In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd

from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import eval_is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.testing import *
from quantfreedom.plotting.strat_dash import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

prices = pd.read_hdf("../data/bigtest.hd5")

rsi_ind = from_talib(
    func_name='rsi',
    prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30, 50],
)
entries = eval_is_below(
    want_to_evaluate=rsi_ind,
    user_args=[20,30,40, 50, 60],
    cand_ohlc='close',
)

In [3]:
prices['LTCUSDT']

candle_info,open,high,low,close
open_time,,,,
2020-10-01 00:00:00,NaN,NaN,NaN,NaN
2020-10-01 00:30:00,NaN,NaN,NaN,NaN
2020-10-01 01:00:00,NaN,NaN,NaN,NaN
2020-10-01 01:30:00,NaN,NaN,NaN,NaN
2020-10-01 02:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2021-04-14 21:30:00,269.26,281.37,269.11,280.82
2021-04-14 22:00:00,280.82,282.02,278.24,278.49
2021-04-14 22:30:00,278.49,281.31,276.64,277.50


In [4]:
entries['LTCUSDT']

RSI_timeperiod          15                                 30                \
RSI_is_below            20     30     40     50     60     20     30     40   
open_time                                                                     
2020-10-01 00:00:00  False  False  False  False  False  False  False  False   
2020-10-01 00:30:00  False  False  False  False  False  False  False  False   
2020-10-01 01:00:00  False  False  False  False  False  False  False  False   
2020-10-01 01:30:00  False  False  False  False  False  False  False  False   
2020-10-01 02:00:00  False  False  False  False  False  False  False  False   
...                    ...    ...    ...    ...    ...    ...    ...    ...   
2021-04-14 21:30:00  False  False  False  False  False  False  False  False   
2021-04-14 22:00:00  False  False  False  False  False  False  False  False   
2021-04-14 22:30:00  False  False  False  False   True  False  False  False   
2021-04-14 23:00:00  False  False  False  False  False  False  False  False   
2021-04-14 23:30:00  False  False  False  False  False  False  False  False   

RSI_timeperiod                        50                              
RSI_is_below            50     60     20     30     40     50     60  
open_time                                                             
2020-10-01 00:00:00  False  False  False  False  False  False  False  
2020-10-01 00:30:00  False  False  False  False  False  False  False  
2020-10-01 01:00:00  False  False  False  False  False  False  False  
2020-10-01 01:30:00  False  False  False  False  False  False  False  
2020-10-01 02:00:00  False  False  False  False  False  False  False  
...                    ...    ...    ...    ...    ...    ...    ...  
2021-04-14 21:30:00  False   True  False  False  False  False   True  
2021-04-14 22:00:00  False   True  False  False  False  False   True  
2021-04-14 22:30:00  False   True  False  False  False  False   True  
2021-04-14 23:00:00  False   True  False  False  False  False   True  
2021-04-14 23:30:00  False   True  False  False  False  False   True  

[9408 rows x 15 columns]

In [ ]:
strat_results_df, settings_results_df = backtest_df_only(
    prices=prices,
    entries=entries,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(1, 7.1, 1),
    risk_rewards=np.arange(1, 10.1, .5),
    size_pct=1.,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(1, 7.1, .5),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(1, 7.1, .5),
    tsl_when_pct_from_avg_entry=np.arange(1, 7.1, .5),
    # gains_pct_filter=100,
)

In [ ]:
strat_results_df.head(50)

In [ ]:
settings_results_df.iloc[:,1251]

In [ ]:
order_records = simulate_up_to_6_nb(
    prices=prices['BTCUSDT'].values,
    entries=entries.iloc[:,[5]].values,
    equity=1000.0,
    fee_pct=0.06,
    mmr=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=5,
    risk_rewards=2,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_pcts_init=5,
    tsl_trail_by_pct=3,
    tsl_when_pct_from_avg_entry=3,
    # gains_pct_filter=100,
)

In [ ]:
entries[[("BTCUSDT", 30, 50)]]

In [ ]:
rsi_ind[[('BTCUSDT', 30)]]

In [ ]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind[[('BTCUSDT', 30)]],
        "entries1": entries[[("BTCUSDT", 30, 50)]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=prices.BTCUSDT,
    order_records=order_records,
)